## 第三部分：会员用户画像
流程如下：
####  1 构建会员用户基本特征标签（性别xb，年龄age，会员积分jf）
####  2 构建会员用户业务特征标签（入会时长rhsc--新、老会员，消费水平--高中低等消费）
####  3 构建会员用户偏好特征标签（购物时间的偏好，购物季节偏好，购物类型的偏好）
####  4 建立用户画像


### 3.0 导入必要的库和数据

In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2020-07-19

@author: 豆奶
"""
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']

In [2]:
data_task1 = pd.read_csv(r"../data/task1.csv")
member = data_task1[['kh', 'xb', 'csrq', 'jf', 'djsj', 'je', 'dtime', 'spmc']]

print(member.shape)
print('处理前缺失值数目是：\n', member.isnull().sum())
member.dropna(inplace=True)
print('\n处理后缺失值数目是：\n', member.isnull().sum())
print(member.shape)

member.loc[:,'dtime'] = pd.to_datetime(member.loc[:,'dtime'],infer_datetime_format=True) # 将年份转为datatime格式 
member.loc[:,'csrq'] = pd.to_datetime(member.loc[:,'csrq'],infer_datetime_format=True, errors="coerce")   # 将年份转为datatime格式 
member.loc[:,'djsj'] = pd.to_datetime(member.loc[:,'djsj'],infer_datetime_format=True) # 将年份转为datatime格式 

d:\python3.7.5\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1971886, 8)
处理前缺失值数目是：
 kh        986811
xb       1341367
csrq     1341367
jf       1135516
djsj     1341367
je        148705
dtime     148705
spmc      148705
dtype: int64


d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



处理后缺失值数目是：
 kh       0
xb       0
csrq     0
jf       0
djsj     0
je       0
dtime    0
spmc     0
dtype: int64
(481814, 8)


d:\python3.7.5\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### 年龄、入会时长 、购物时间偏好、购物季节偏好

In [3]:
now_year = datetime.datetime.today().year      #当前的年份
member['age'] = now_year-member.csrq.dt.year   # 年龄
member['rhsc'] = now_year-member.djsj.dt.year  # 入会时长

member = member.drop(member[(member.age > 100)].index)   #  删除异常值与缺失值
member.dropna(inplace=True)

d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
def get_gwsjp( x ):   # 购物时间偏好
    time_6 = datetime.datetime.strptime('6:00','%H:%M').time()
    time_12 = datetime.datetime.strptime('12:00','%H:%M').time()
    time_18 = datetime.datetime.strptime('18:00','%H:%M').time()
    xtime = x.timetz()
    #xtime = x.dtime.dt.time
    if xtime<time_6:
        return '凌晨'
    elif xtime<time_12:
        return '上午'
    elif xtime<time_18:
        return '下午'
    else:
        return '晚上'
    
member['gwsjp'] = member['dtime'].map(get_gwsjp)

In [5]:
def get_gwjjp( x ):   # 购物季节偏好
    xmonth = x.month
    #xtime = x.dtime.dt.time
    if 3<=xmonth<6:
        return '春'
    elif 6<=xmonth<9:
        return '夏'
    elif 9<=xmonth<12:
        return '秋'
    else:
        return '冬'
    
member['gwjjp'] = member['dtime'].map(get_gwjjp)
member.head()

,kh,xb,csrq,jf,djsj,je,dtime,spmc,age,rhsc,gwsjp,gwjjp
2,a37cc182,0.0,1967-02-17,297.08,2004-12-31 21:24:34.216,297.07,2016-09-25 10:49:14.016,WMF D无,53.0,16,上午,秋
3,a37cc182,0.0,1967-02-17,598.00,2004-12-31 21:24:34.216,598.00,2017-02-28 17:17:35.533,双立人商品 F无,53.0,16,下午,冬
7,34c2dc93,0.0,1986-07-18,260.00,2007-04-19 00:00:00.000,260.00,2017-11-13 13:36:45.656,植村秀三色眼影/眼线液/眉笔.,34.0,13,下午,秋
10,828aa9b4,0.0,1951-12-29,1288.00,2004-09-24 16:39:08.716,1288.00,2017-05-17 11:30:39.080,酷彩F件,69.0,16,上午,春
11,828aa9b4,0.0,1951-12-29,1378.00,2004-09-24 16:39:08.716,1378.00,2017-07-29 18:32:24.720,InsunF件,69.0,16,晚上,夏


In [6]:
# 获取购买总积分、总金额、商品名称、消费时间等数据
data = member.groupby(["kh"], as_index=False).agg({'jf':'sum', 'je':'sum', 'spmc':'unique', 'dtime':'unique'})
print(data.shape, member.shape)
data.head()    

(36503, 5) (439805, 12)


,kh,jf,je,spmc,dtime
0,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:..."
1,000234ad,11850.0,12850.0,"[wacoal 正价\件, 和音E., JuicyCouFureF件, 雅诗兰黛白金级奢宠紧...","[2015-01-02T13:25:52.626000000, 2015-01-02T13:..."
2,000339f1,6141.0,6340.8,"[wacoal 正价\件, 植村秀眉笔4g, 植村秀汗杆/汗芯., 植村秀无色限彩妆盒四格....","[2015-03-30T14:11:49.610000000, 2016-03-18T14:..."
3,0004bad2,3312.0,3312.0,[朗姿A件],[2017-10-26T13:15:32.296000000]
4,00075d60,28672.0,35710.0,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...","[2015-03-27T17:27:00.610000000, 2016-04-21T11:..."


In [7]:
# 合并得到会员的 积分，消费水平，购物类型的偏好，性别，入会时长，购物时间的偏好，购物季节偏好，年龄等数据
label =  pd.merge(data, member.loc[:, ['kh','xb','rhsc', 'age', 'gwsjp', 'gwjjp']],on='kh', how="left")
print(label.shape)
label.head()

(439805, 10)


,kh,jf,je,spmc,dtime,xb,rhsc,age,gwsjp,gwjjp
0,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春
1,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春
2,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春
3,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春
4,000186fa,5267.0,11880.7,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,秋


In [8]:
label.je[label.je<100] = 0
label.je[(100<=label.je)&(label.je<=200)] = 100
label.je[label.je>200] = 200
label.je[label.je==0] = '低消费'
label.je[label.je==100] = '中等消费'
label.je[label.je==200] = '高消费'

d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\python3.7.5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d

In [9]:
# 去除卡号重复的数据
print(label.shape)
label.drop_duplicates(subset=['kh'], inplace=True)
print(label.shape)
label.head()

(439805, 10)
(36503, 10)


,kh,jf,je,spmc,dtime,xb,rhsc,age,gwsjp,gwjjp
0,000186fa,5267.0,高消费,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春
18,000234ad,11850.0,高消费,"[wacoal 正价\件, 和音E., JuicyCouFureF件, 雅诗兰黛白金级奢宠紧...","[2015-01-02T13:25:52.626000000, 2015-01-02T13:...",0.0,11,45.0,下午,冬
30,000339f1,6141.0,高消费,"[wacoal 正价\件, 植村秀眉笔4g, 植村秀汗杆/汗芯., 植村秀无色限彩妆盒四格....","[2015-03-30T14:11:49.610000000, 2016-03-18T14:...",0.0,7,32.0,下午,春
50,0004bad2,3312.0,高消费,[朗姿A件],[2017-10-26T13:15:32.296000000],0.0,7,36.0,下午,秋
51,00075d60,28672.0,高消费,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...","[2015-03-27T17:27:00.610000000, 2016-04-21T11:...",0.0,4,52.0,下午,春


In [10]:
# 获取消费次数
label['xfcs'] = label.apply(lambda x:x['dtime'].shape[0], axis=1)       # 消费次数
label.head()

,kh,jf,je,spmc,dtime,xb,rhsc,age,gwsjp,gwjjp,xfcs
0,000186fa,5267.0,高消费,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...","[2015-05-02T13:14:01.516000000, 2016-09-24T12:...",0.0,4,43.0,下午,春,4
18,000234ad,11850.0,高消费,"[wacoal 正价\件, 和音E., JuicyCouFureF件, 雅诗兰黛白金级奢宠紧...","[2015-01-02T13:25:52.626000000, 2015-01-02T13:...",0.0,11,45.0,下午,冬,7
30,000339f1,6141.0,高消费,"[wacoal 正价\件, 植村秀眉笔4g, 植村秀汗杆/汗芯., 植村秀无色限彩妆盒四格....","[2015-03-30T14:11:49.610000000, 2016-03-18T14:...",0.0,7,32.0,下午,春,10
50,0004bad2,3312.0,高消费,[朗姿A件],[2017-10-26T13:15:32.296000000],0.0,7,36.0,下午,秋,1
51,00075d60,28672.0,高消费,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...","[2015-03-27T17:27:00.610000000, 2016-04-21T11:...",0.0,4,52.0,下午,春,8


### 任务3.4 建立用户画像

In [11]:
label.columns = ['卡号', '积分', '消费水平', '商品名称', 'dtime', '性别', '入会时长', '年龄', '购物时间偏好', '购物季节偏好', '消费次数']
user_portrait = label[['卡号', '积分', '消费水平', '商品名称', '性别', '入会时长', '年龄', '购物时间偏好', '购物季节偏好', '消费次数']]
user_portrait.reset_index(drop=True, inplace=True)
user_portrait.head()

,卡号,积分,消费水平,商品名称,性别,入会时长,年龄,购物时间偏好,购物季节偏好,消费次数
0,000186fa,5267.0,高消费,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...",0.0,4,43.0,下午,春,4
1,000234ad,11850.0,高消费,"[wacoal 正价\件, 和音E., JuicyCouFureF件, 雅诗兰黛白金级奢宠紧...",0.0,11,45.0,下午,冬,7
2,000339f1,6141.0,高消费,"[wacoal 正价\件, 植村秀眉笔4g, 植村秀汗杆/汗芯., 植村秀无色限彩妆盒四格....",0.0,7,32.0,下午,春,10
3,0004bad2,3312.0,高消费,[朗姿A件],0.0,7,36.0,下午,秋,1
4,00075d60,28672.0,高消费,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...",0.0,4,52.0,下午,春,8


In [12]:
user_portrait.性别 = '女' if [user_portrait.性别=='女'] else '男'
# user_portrait.性别[user_portrait.性别==0.0] = '女'
# user_portrait.性别[user_portrait.性别==1.0] = '男'
user_portrait.head()

d:\python3.7.5\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
d:\python3.7.5\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,卡号,积分,消费水平,商品名称,性别,入会时长,年龄,购物时间偏好,购物季节偏好,消费次数
0,000186fa,5267.0,高消费,"[雅诗兰黛肌初赋活原生液., 雅诗兰黛密集焕白遮瑕笔., 雅诗兰黛晶透沁白淡斑精粹水., 雅...",女,4,43.0,下午,春,4
1,000234ad,11850.0,高消费,"[wacoal 正价\件, 和音E., JuicyCouFureF件, 雅诗兰黛白金级奢宠紧...",女,11,45.0,下午,冬,7
2,000339f1,6141.0,高消费,"[wacoal 正价\件, 植村秀眉笔4g, 植村秀汗杆/汗芯., 植村秀无色限彩妆盒四格....",女,7,32.0,下午,春,10
3,0004bad2,3312.0,高消费,[朗姿A件],女,7,36.0,下午,秋,1
4,00075d60,28672.0,高消费,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...",女,4,52.0,下午,春,8


In [13]:
# 定义获取用户画像的函数
def get_user_message(kh):
    message = label[label.卡号==kh]
    message.head
    for i in message:
        if(i!='商品名称' and i!='dtime'):
            print(i,': ', message.iloc[0][i])

In [14]:
# 测试
kh = '00075d60'
get_user_message(kh)
label[label.卡号==kh]

卡号 :  00075d60
积分 :  28672.0
消费水平 :  高消费
性别 :  0.0
入会时长 :  4
年龄 :  52.0
购物时间偏好 :  下午
购物季节偏好 :  春
消费次数 :  8


,卡号,积分,消费水平,商品名称,dtime,性别,入会时长,年龄,购物时间偏好,购物季节偏好,消费次数
51,00075d60,28672.0,高消费,"[DOUBLE SFANDARDF., 希思黎全日呵护精华乳., 希思黎焕白亮采面膜., 希...","[2015-03-27T17:27:00.610000000, 2016-04-21T11:...",0.0,4,52.0,下午,春,8
